### Phenology metric calculations

In [1]:
import os
import ee
ee.Initialize()
import pandas as pd
from IPython.display import JSON
import geemap

### Functions

In [3]:
# add dn for product aggregated into 8 days
def add_dn_date(img,beginDate=None,n=None,IncludeYear=False):
    if beginDate is None:
        beginDate = img.get('system:time_start')
    else:
        beginDate = beginDate
    if IncludeYear is False:
        IncludeYear = True
    if n is None:
        n = 8
    beginDate = ee.Date(beginDate)
    year  = beginDate.get('year')
    month = beginDate.get('month')
    diff  = beginDate.difference(ee.Date.fromYMD(year, 1, 1), 'day').add(1)
    dn    = diff.subtract(1).divide(n).floor().add(1).int()
    yearstr  = year.format('%d') 
    dn = dn.format('%02d')
    return ee.Image(img).set('system:time_start', beginDate.millis()).set('date', beginDate.format('yyyy-MM-dd')).set('Year', yearstr).set('Month',beginDate.format('MM')).set('YearMonth', beginDate.format('YYYY-MM')).set('dn', dn)

# wrapper function for add_dn_date
def add_dn_date_all(Year,days):
    def wrapper(image0):
        tmp = add_dn_date(img = image0,IncludeYear=Year,n = days)
        return tmp
    return (wrapper)

# add NDVI to data
def addNDVI(image):
    #image = image.updateMask(MakMarco.eq(1))
    return image.addBands(image.normalizedDifference(['sur_refl_b02','sur_refl_b01']).rename('NDVI')).float()

# function for extracting quality bits
def getQABits(image, start, end, mascara):
    # Compute the bits we need to extract.
    pattern = 0
    for i in range(start,end+1):
        pattern += 2**i
    # Return a single band image of the extracted QA bits, giving the     band a new name.
    return image.select([0], [mascara]).bitwiseAnd(pattern).rightShift(start)

# mask out low quality pixels (based on flags)
def maskPixels(image0):
    #Select the QA band
    QA = image0.select('StateQA')
    # Get the land_water_flag bits
    landWaterFlag = getQABits(QA, 3, 5, 'land_water_flag')
    #Get the cloud_state bits and find cloudy areas.
    cloud = getQABits(QA, 0, 1, 'cloud_state').expression("b(0) == 1 || b(0) == 2")
    # Get the cloud_shadow bit
    cloudShadows = getQABits(QA, 2, 2, 'cloud_shadow')
    # Get the Pixel is adjacent to cloud bit
    cloudAdjacent = getQABits(QA, 13, 13, 'cloud_adj')
    # Get the internal cloud flag
    cloud2 = getQABits(QA, 10, 10, 'cloud_internal')
    # Get the internal fire flag
    fire = getQABits(QA, 11, 11, 'fire_internal')
    # Get the MOD35 snow/ice flag
    snow1 = getQABits(QA, 12, 12, 'snow_MOD35')
    # Get the internal snow flag
    snow2 = getQABits(QA, 15, 15, 'snow_internal')
    # create mask
    mask = landWaterFlag.eq(1).And(cloud.Not()).And(cloudShadows.Not()).And(cloudAdjacent.Not()).And(cloud2.Not()).And(fire.Not()).And(snow1.Not()).And(snow2.Not())
    return image0.updateMask(mask) 

# combine two images (using match image)
def imgstobands(image):
    return image.addBands(image.get('match'))

# identify images which should not be used 
def smoothed_redmin(image):
    tmp = image.reproject(crs = 'SR-ORG:6974',scale=463.3127165275).\
        reduceResolution(ee.Reducer.min(),bestEffort = False, maxPixels = 65536).\
        reproject(crs = ee.Projection('EPSG:4326').scale(0.05 , 0.05)).updateMask(1)
    return tmp.select('NDVI').rename('NDVI_5km')


# count pixels 
def smoothed_redcount1(image):
    tmp = image.reproject(crs = 'SR-ORG:6974',scale=463.3127165275).\
        reduceResolution(ee.Reducer.count(),bestEffort = False, maxPixels = 65536).\
        reproject(crs = ee.Projection('EPSG:4326').scale(0.05 , 0.05)).updateMask(1)
    return tmp.select('NDVI').rename('NDVI_5km_X')


# count pixels 
def smoothed_redcount2(image):
    tmp = image.reproject(crs = 'SR-ORG:6974',scale=463.3127165275).\
        reduceResolution(ee.Reducer.count(),bestEffort = False, maxPixels = 65536).\
        reproject(crs = ee.Projection('EPSG:4326').scale(0.05 , 0.05)).updateMask(1)
    return tmp.select('NDVI').rename('NDVI_5km_Y')

# filter smoothed data: fill in images with 0s. This gives all the pixels theoretically available
def filt_smoothed_c(image):
    image = image.select('NDVI')
    image = image.unmask()
    image = image.where(image.eq(0),0)
    return image.updateMask(count_valid.gte(15)).updateMask(forestmask.eq(1))

# filter smoothed data: this keeps usable pixels
def filt_smoothed2_c(image):
    image = image.select('NDVI')
    #image = image.unmask()
    #image = image.where(image.eq(0),-9999)
    return image.updateMask(count_valid.gte(15)).updateMask(forestmask.eq(1))

# calculate delta
def deltac(image):
    return image.select('NDVI_5km_X').subtract(image.select('NDVI_5km_Y')).rename('delta')

# filter images by delta
def delta_filt(image):
    return image.select('NDVI').updateMask(ee.Image(image.get('match')).lt(10))


def smooth_func(image): 
    collection = ee.ImageCollection.fromImages(image.get('images'))
    return ee.Image(image).addBands(collection.mean().rename(['mean']))

#def clim5y(month):
#    month = ee.String(month)
#    seqNDVI = MOD09ndviY.filterMetadata('dn', 'equals',month)
#    return seqNDVI.median().copyProperties(seqNDVI.first(), ['system:time_start','system:time_end','dn','Month'])


def clim5y(month):
    seqNDVI1 = MOD09ndviY1.filterMetadata('dn', 'equals',month).first()
    # select image from year of interest
    seqNDVI2 = MOD09ndviY2.filterMetadata('dn', 'equals',month) 
    inversemask = seqNDVI1.gt(-1).selfMask()
    median = seqNDVI2.map(lambda image: image.updateMask(inversemask.unmask().Not())).median()
    final = ee.ImageCollection.fromImages([seqNDVI1,median]).median()
    return final.copyProperties(seqNDVI1, ['system:time_start','system:time_end','dn','Month'])

def clim5y1(month):
    seqNDVI1 = MOD09ndviY1.filterMetadata('dn', 'equals',month).first() # focal year
    seqNDVI2 = MOD09ndviY2.filterMetadata('dn', 'equals',month).median() # five years ahead
    # combine together
    nodata = -9999
    mask = seqNDVI1.mask()
    img = seqNDVI1.unmask(nodata)
    img = img.where(mask.Not(), seqNDVI2)
    img = img.updateMask(img.neq(nodata))
    return img #img.copyProperties(seqNDVI1, ['system:time_start','system:time_end','dn','Month'])



def tempfilt(image):
    mask = ee.Image(ee.List(image.get('match')).get(0))
    return image.updateMask(mask)
                           
# function for accumulating NDVI
def accumulate(image,list):
    # Get the latest cumulative NDVI of the list with
    # get(-1).  Since the type of the list argument to the function is unknown,
    # it needs to be cast to a List.  Since the return type of get() is unknown,
    # cast it to Image.
    previous = ee.Image(ee.List(list).get(-1)).toFloat().unmask()
    # Add the current anomaly to make a new cumulative NDVI image and Propagate metadata to the new image.
    added = image.unmask().toFloat().add(previous).toFloat().set('system:time_start', image.get('system:time_start'))
    return ee.List(list).add(added)

# cumulative normalized
def cum_dividelast(image):
    tmp = image.divide(last).multiply(Cumulative_mean_at_5km)
    return tmp

# calculate deviations
def deviations_calc(image):
    tmp = image.select('NDVI').setDefaultProjection(crs = 'SR-ORG:6974',scale = 463.3127165275).reduceResolution(ee.Reducer.stdDev(), bestEffort = False, maxPixels = 150).reproject(ee.Projection('EPSG:4326').scale(0.05, 0.05)).updateMask(1).rename('stdDev')
    return tmp


def addTimeBand(img):
    ## make sure mask is consistent ##
    mask = img.mask()
    time = img.metadata('system:time_start').rename("time").mask(mask)
    return img.addBands(time)


def replace_mask(img, newimg, nodata):
    if frame is None:
        nodata = 8
    # var con = img.mask();
    # var res = img., NODATA
    mask = img.mask()
    # The only nsolution is unmask & updatemask */
    img = img.unmask(nodata)
    img = img.where(mask.Not(), newimg)
    img = img.updateMask(img.neq(nodata))
    return img
    
def linearInterp(imgcol,frame = None,nodata = None):
    if frame is None:
        frame = 32
    if nodata is None:
        nodata = 0
    timestart   = 'system:time_start'
    imgcol = imgcol.map(addTimeBand)
    
    # We'll look for all images up to 32 days away from the current image (if frame not provided!)
    maxDiff = ee.Filter.maxDifference(frame * (1000*60*60*24), timestart, None, timestart)
    
    #cond    = {'leftField':timestart, 'rightField':timestart}
    # Images after, sorted in descending order (so closest is last).
    #var f1 = maxDiff.and(ee.Filter.lessThanOrEquals(time, null, time))
    f1 = ee.Filter.And(maxDiff, ee.Filter.lessThanOrEquals(leftField = timestart,rightField = timestart))
    c1 = ee.Join.saveAll(matchesKey = 'after', ordering = timestart, ascending = False).apply(imgcol, imgcol, f1)
    # Images before, sorted in ascending order (so closest is last).
    # var f2 = maxDiff.and(ee.Filter.greaterThanOrEquals(time, null, time))
    
    f2 = ee.Filter.And(maxDiff, ee.Filter.greaterThanOrEquals(leftField = timestart,rightField = timestart))
    c2 = ee.Join.saveAll(matchesKey = 'before', ordering = timestart, ascending = True).apply(c1, imgcol, f2)
    
    # interpolation 
    def func_its(img):
        img = ee.Image(img)
        before = ee.ImageCollection.fromImages(ee.List(img.get('before'))).mosaic()
        after  = ee.ImageCollection.fromImages(ee.List(img.get('after'))).mosaic()
        img = img.set('before', {}).set('after', {})
        
        # constrain after or before no NA values, confirm linear Interp having result
        before = replace_mask(before, after, nodata)
        after  = replace_mask(after , before, nodata)
        
        # Compute the ratio between the image times.
        x1 = before.select('time').double()
        x2 = after.select('time').double()
        now = ee.Image.constant(img.date().millis()).double()
        ratio = now.subtract(x1).divide(x2.subtract(x1))  # this is zero anywhere x1 = x2
        # Compute the interpolated image.
        before = before.select(0); #remove time band now
        after  = after.select(0)
        img    = img.select(0)
        qc = img.mask().Not().rename('qc')
        interp = replace_mask(img, interp, nodata)
        
        # Map.addLayer(interp, {}, 'interp')
        return interp.addBands(qc).copyProperties(img, img.propertyNames())
    interpolated = ee.ImageCollection(c2.map(func_its))
    return interpolated


### Load in collections and required images

In [4]:
# load collections and required images
collection = ee.ImageCollection('MODIS/006/MOD09A1').filterDate('2000-01-01', '2020-12-31')
forestmask = ee.Image('users/marcogirardello/phenoutils/mask_unchanged_500m')
smallgrid = ee.FeatureCollection('users/marcogirardello/phenoutils/grid_export_phenology22')
terracl = ee.Image("users/marcogirardello/phenoutils/terraclimate_months")

#### Create monthly binary masks based on TERRACLIMATE long-term averages

In [5]:
# convert into image collection
terracl1 = ee.ImageCollection.fromImages(terracl.bandNames().map(lambda name: terracl.select([name])))
# add month property taken from bands
terracl2 = terracl1.map(lambda image: image.set(ee.Dictionary({'Month': ee.String(image.bandNames().get(0)).slice(2)})))
# rename bands into b1
terracl3 = terracl2.map(lambda image: image.rename('avgtemp'))
# create binary map showing where temperature is greater than 0
terraclmonths = terracl3.map(lambda image: image.gt(0).copyProperties(image,['Month']))

In [5]:
terraclimbin = terraclmonths.map(lambda image: image.where(image.gt(0),1))
# sum of pixel count across 12 months where temperature > 0 °C  
ImgCountDistinct = terraclimbin.reduce(ee.Reducer.sum())
# create pixel specific filter: i.e. number of observations that one needs to have in the time series for cumulating
NumberOfMinImagePerYear = ImgCountDistinct.multiply(2)

### <span style="color:blue">Pre-processing step 1: filtering data by quality flags and calculated NDVI.</span>
This include snow, cloud, fire, cloud shadows and the land/water mask

In [6]:
# end and start date of period of interest
start_date = ee.Date.fromYMD(2001, 1, 1)
end_date   = ee.Date.fromYMD(2020, 12, 31)

In [7]:
# add dn
collection = collection.map(add_dn_date_all(Year = False, days = 8))

In [8]:
# mask out crap pixels
MOD09masked = collection.filterDate(start_date, end_date).map(maskPixels)

In [9]:
# add NDVI as a new band
MOD09ndvi = MOD09masked.map(addNDVI).select('NDVI')

In [11]:
# end and start date of period of interest
#start_date = ee.Date.fromYMD(2003, 1, 1)
#end_date   = ee.Date.fromYMD(2003, 12, 31)
#imgcol = MOD09ndvi.filterDate(start_date, end_date)

### <span style="color:blue">Main calculations</span>

In [12]:
tmpseas = ["%02d" % x for x in list(range(1, 46+1))]
tmpseas1 = ee.List(tmpseas)

In [22]:
# test environment variable (JUST RELEVANT FOR EXPORT!)
%env year=2003
year = int(os.environ["year"])

env: year=2003


In [14]:
# climatology
#yearp5 = ee.Number(year).add(5)
#start_date = ee.Date.fromYMD(year,1,1)
#end_date = ee.Date.fromYMD(yearp5,12,31)
#MOD09ndviY = MOD09ndvi.filterDate(start_date, end_date)
#MinNDVI = MOD09ndviY.min()
#meanNDVI = MOD09ndviY.mean()

#yearp5 = ee.Number(year).add(5)
#start_date = ee.Date.fromYMD(year,1,1)
#end_date1 = ee.Date.fromYMD(year,12,31)
#end_date2 = ee.Date.fromYMD(yearp5,12,31)
#MOD09ndviY1 = MOD09ndvi.filterDate(start_date, end_date1)
#MOD09ndviY2 = MOD09ndvi.filterDate(start_date, end_date2)

# needed for interpolation
frame  = 8*8
nodata = -9999

# filter year range only for focal year
start_date = ee.Date.fromYMD(year,1,1)
end_date = ee.Date.fromYMD(year,12,31)
MOD09ndviY1_tmp = MOD09ndvi.filterDate(start_date, end_date)

# linear interpolation for focal years
MOD09ndviY1 = linearInterp(MOD09ndviY1_tmp, frame, nodata).select('NDVI')

if year > 2015:
    # temporary year set to 2015
    yeartmp = 2015
    # filter for additional four years
    MOD09ndviY2 = MOD09ndvi.filterDate(ee.Date.fromYMD(yeartmp+1,1,1),ee.Date.fromYMD(yeartmp+5,12,31))
else: 
    # filter for additional four years
    MOD09ndviY2 = MOD09ndvi.filterDate(ee.Date.fromYMD(year+1,1,1),ee.Date.fromYMD(year+5,12,31))

# climatology 5 years (monthly composites)
seasons2 = ee.ImageCollection.fromImages(tmpseas1.map(clim5y1)) 

# climatology 5 years (monthly composites)
#seasons = ee.ImageCollection.fromImages(tmpseas1.map(clim5y)) 

# linear interpolation
#frame  = 8*10
#nodata = -9999
#seasons2 = linearInterp(MOD09ndviY1, frame, nodata)
#count_valid = seasons2.select('qc').count()
#seasons3 = linearInterp(seasons, frame, nodata)

# -------- Climatic filter
# join with terraclimate data
jfilter = ee.Filter.equals(leftField = 'Month',rightField = 'Month')
simplej = ee.Join.inner()
seasons2j = simplej.apply(seasons2,terraclmonths,jfilter)
# add temperature mask as a new band
seasons2 = ee.ImageCollection(seasons2j.map(lambda joinresult: ee.Image(joinresult.get('primary')).addBands(joinresult.get('secondary'))))
# filter for temperatures bigger than > 0 °C
seasons2 = seasons2.map(lambda image: image.updateMask(image.select('avgtemp').gt(0)))

count_valid = seasons2.select('NDVI').count()

smoothed  = seasons2.map(lambda image: image.updateMask(count_valid.gte(15)).updateMask(forestmask.eq(1)))

ct = smoothed.select('NDVI').count()

ctmax  = ct.setDefaultProjection(crs = 'SR-ORG:6974',scale = 463.3127165275).\
                          reduceResolution(ee.Reducer.max(), bestEffort = False, maxPixels = 150).\
                          reproject(ee.Projection('EPSG:4326').scale(0.05, 0.05)).updateMask(1) 

ctmean  = ct.setDefaultProjection(crs = 'SR-ORG:6974',scale = 463.3127165275).\
                          reduceResolution(ee.Reducer.mean(), bestEffort = False, maxPixels = 150).\
                          reproject(ee.Projection('EPSG:4326').scale(0.05, 0.05)).updateMask(1) 
# create count filter
#imgcount = smoothed.count()
#sdcount = imgcount.select('NDVI').reproject(crs = 'SR-ORG:6974',scale=463.3127165275).\
#reduceResolution(ee.Reducer.stdDev(),bestEffort = False, maxPixels = 65536).\
#reproject(crs = ee.Projection('EPSG:4326').scale(0.05 , 0.05)).updateMask(1)


#Get the timestamp from the most recent image in the reference collection.
time0 = smoothed.first().get('system:time_start')

# Rename the first band 'NDVI'.
first = ee.List([ee.Image(0).set('system:time_start', time0).select([0],['NDVI']).toFloat().set(ee.Dictionary({'Month': '01'}))])

# Since the return type of iterate is unknown, it needs to be cast to a List.
cumulative = ee.ImageCollection(ee.List(smoothed.iterate(accumulate, first)))

cumulative = cumulative.map(lambda image: image.select('NDVI').updateMask(image.select('NDVI').gt(0)))

# normalise
last = cumulative.sort('system:time_start', False).first()
last = last.updateMask(last.gte(3.5)) # Changed from 6 to 3.5!!!

# mean of cumulative values
Cumulative_mean = last.divide(46)
Cumulative_mean_at_5km = Cumulative_mean.setDefaultProjection(crs = 'SR-ORG:6974',scale = 463.3127165275).reduceResolution(ee.Reducer.mean(), bestEffort = False, maxPixels = 150).reproject(ee.Projection('EPSG:4326').scale(0.05, 0.05)).updateMask(1) 

# cumulative map normalised
cumulativeNorm = cumulative.map(cum_dividelast)

# deviations (these are described in the document sent by Alessandro)
cumulativeStd10 = cumulativeNorm.map(deviations_calc)

# calculate the mean of the deviations
pixelmask = ctmean.eq(ctmax)
prop = ctmean.divide(ctmax)
#cumulativeStd10_mean = cumulativeStd10.filterBounds(smallgrid1).mean().multiply(10000).updateMask(prop.gte(0.98))


### <span style="color:blue">Export data</span>

In [13]:
%env minpoly=1
%env maxpoly=40
minpoly = int(os.environ["minpoly"])
maxpoly = int(os.environ["maxpoly"])

env: minpoly=1
env: maxpoly=40


In [ ]:
polyl = list(range(minpoly,maxpoly+1))

In [52]:
# Remember resolution!
for poly in polyl:
    #print(poly)
    # filter a given square
    onesquare = smallgrid.filterMetadata('polyID','equals',poly)
    roi1 = ee.Geometry.Polygon(onesquare.geometry().getInfo().get('coordinates'))
    roi1 = roi1.bounds()
    # create filtered collection
    cumulativeStd10_mean = cumulativeStd10.filterBounds(onesquare).mean().multiply(10000).updateMask(prop.gte(0.98))
    # polygon number
    polyname = 'Y_'+str(year)+'_'+'poly_'+str(poly)+'_stripe'
    # export tile
    filename = 'Y_'+str(year)+'_'+'poly_'+str(poly)+'_stripe'
    task = ee.batch.Export.image.toCloudStorage(image = cumulativeStd10_mean,description = polyname,
                                     bucket = 'tmpmarcog1',fileNamePrefix = filename,
                                     scale = 463.3127165275,fileFormat='GeoTIFF',
                                    skipEmptyTiles = True, crs ='EPSG:4326',maxPixels=1e13,
                                    region = roi1)
    task.start()

AttributeError: 'dict' object has no attribute 'filterMetadata'

In [ ]:
smallgrid = ee.FeatureCollection('users/marcogirardello/phenoutils/grid_export_phenology22')

In [5]:
Map.addLayer(smallgrid)

In [4]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [ ]:
#Removing gs://tmpmarcog1/Y_2003_poly_1.tif...
#Removing gs://tmpmarcog1/Y_2003_poly_10.tif...
#Removing gs://tmpmarcog1/Y_2003_poly_100.tif...
#Removing gs://tmpmarcog1/Y_2003_poly_100_8obs.tif...
#Removing gs://tmpmarcog1/Y_2003_poly_101.tif...
#Removing gs://tmpmarcog1/Y_2003_poly_101_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_102.tif...
Removing gs://tmpmarcog1/Y_2003_poly_102_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_103.tif...
Removing gs://tmpmarcog1/Y_2003_poly_103_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_104.tif...
Removing gs://tmpmarcog1/Y_2003_poly_104_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_107.tif...
Removing gs://tmpmarcog1/Y_2003_poly_107_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_108.tif...
Removing gs://tmpmarcog1/Y_2003_poly_108_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_109.tif...
Removing gs://tmpmarcog1/Y_2003_poly_109_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_10_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_11.tif...
Removing gs://tmpmarcog1/Y_2003_poly_110.tif...
Removing gs://tmpmarcog1/Y_2003_poly_110_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_111.tif...
Removing gs://tmpmarcog1/Y_2003_poly_111_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_112_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_112.tif...
Removing gs://tmpmarcog1/Y_2003_poly_113.tif...
Removing gs://tmpmarcog1/Y_2003_poly_113_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_114.tif...
Removing gs://tmpmarcog1/Y_2003_poly_114_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_115.tif...
Removing gs://tmpmarcog1/Y_2003_poly_115_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_116.tif...
Removing gs://tmpmarcog1/Y_2003_poly_116_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_117.tif...
Removing gs://tmpmarcog1/Y_2003_poly_117_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_118.tif...
Removing gs://tmpmarcog1/Y_2003_poly_119.tif...
Removing gs://tmpmarcog1/Y_2003_poly_11_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_118_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_120.tif...
Removing gs://tmpmarcog1/Y_2003_poly_119_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_12.tif...
Removing gs://tmpmarcog1/Y_2003_poly_121.tif...
Removing gs://tmpmarcog1/Y_2003_poly_120_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_121_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_122.tif...
Removing gs://tmpmarcog1/Y_2003_poly_122_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_123.tif...
Removing gs://tmpmarcog1/Y_2003_poly_124.tif...
Removing gs://tmpmarcog1/Y_2003_poly_124_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_126_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_128.tif...
Removing gs://tmpmarcog1/Y_2003_poly_126.tif...
Removing gs://tmpmarcog1/Y_2003_poly_127.tif...
Removing gs://tmpmarcog1/Y_2003_poly_127_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_128_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_123_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_129.tif...
Removing gs://tmpmarcog1/Y_2003_poly_129_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_12_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_135.tif...
Removing gs://tmpmarcog1/Y_2003_poly_135_8obs.tif...                            
Removing gs://tmpmarcog1/Y_2003_poly_136.tif...                                 
Removing gs://tmpmarcog1/Y_2003_poly_130.tif...                                 
Removing gs://tmpmarcog1/Y_2003_poly_137.tif...                                 
Removing gs://tmpmarcog1/Y_2003_poly_137_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_136_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_138.tif...                                 
Removing gs://tmpmarcog1/Y_2003_poly_131.tif...
Removing gs://tmpmarcog1/Y_2003_poly_13.tif...                                  
Removing gs://tmpmarcog1/Y_2003_poly_130_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_131_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_132.tif...                                 
Removing gs://tmpmarcog1/Y_2003_poly_133.tif...                                 
Removing gs://tmpmarcog1/Y_2003_poly_132_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_138_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_133_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_139.tif...                                 
Removing gs://tmpmarcog1/Y_2003_poly_140.tif...                                 
Removing gs://tmpmarcog1/Y_2003_poly_13_8obs.tif...                             
Removing gs://tmpmarcog1/Y_2003_poly_141.tif...                                 
Removing gs://tmpmarcog1/Y_2003_poly_141_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_139_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_140_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_14.tif...
Removing gs://tmpmarcog1/Y_2003_poly_142.tif...
Removing gs://tmpmarcog1/Y_2003_poly_142_8obs.tif...                            
Removing gs://tmpmarcog1/Y_2003_poly_143_8obs.tif...                            
Removing gs://tmpmarcog1/Y_2003_poly_143.tif...
Removing gs://tmpmarcog1/Y_2003_poly_144.tif...
Removing gs://tmpmarcog1/Y_2003_poly_145.tif...
Removing gs://tmpmarcog1/Y_2003_poly_144_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_146.tif...
Removing gs://tmpmarcog1/Y_2003_poly_145_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_146_8obs.tif...                            
Removing gs://tmpmarcog1/Y_2003_poly_147.tif...
Removing gs://tmpmarcog1/Y_2003_poly_147_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_149_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_15.tif...
Removing gs://tmpmarcog1/Y_2003_poly_150.tif...
Removing gs://tmpmarcog1/Y_2003_poly_149.tif...
Removing gs://tmpmarcog1/Y_2003_poly_14_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_150_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_151_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_151.tif...
Removing gs://tmpmarcog1/Y_2003_poly_153.tif...
Removing gs://tmpmarcog1/Y_2003_poly_152_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_152.tif...
Removing gs://tmpmarcog1/Y_2003_poly_153_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_154_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_154.tif...
Removing gs://tmpmarcog1/Y_2003_poly_155.tif...
Removing gs://tmpmarcog1/Y_2003_poly_155_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_156_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_156.tif...
Removing gs://tmpmarcog1/Y_2003_poly_157.tif...
Removing gs://tmpmarcog1/Y_2003_poly_157_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_158.tif...                                 
Removing gs://tmpmarcog1/Y_2003_poly_158_8obs.tif...                            
Removing gs://tmpmarcog1/Y_2003_poly_159.tif...
Removing gs://tmpmarcog1/Y_2003_poly_159_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_160.tif...                                 
Removing gs://tmpmarcog1/Y_2003_poly_15_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_161.tif...
Removing gs://tmpmarcog1/Y_2003_poly_161_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_160_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_162.tif...
Removing gs://tmpmarcog1/Y_2003_poly_162_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_163.tif...                                 
Removing gs://tmpmarcog1/Y_2003_poly_163_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_164.tif...                                 
Removing gs://tmpmarcog1/Y_2003_poly_164_8obs.tif...                            
Removing gs://tmpmarcog1/Y_2003_poly_167.tif...                                 
Removing gs://tmpmarcog1/Y_2003_poly_165.tif...
Removing gs://tmpmarcog1/Y_2003_poly_17.tif...
Removing gs://tmpmarcog1/Y_2003_poly_165_8obs.tif...                            
Removing gs://tmpmarcog1/Y_2003_poly_169.tif...
Removing gs://tmpmarcog1/Y_2003_poly_170_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_169_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_171.tif...
Removing gs://tmpmarcog1/Y_2003_poly_167_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_171_8obs.tif...                            
Removing gs://tmpmarcog1/Y_2003_poly_168.tif...
Removing gs://tmpmarcog1/Y_2003_poly_168_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_170.tif...
Removing gs://tmpmarcog1/Y_2003_poly_166.tif...
Removing gs://tmpmarcog1/Y_2003_poly_166_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_180_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_17_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_180.tif...
Removing gs://tmpmarcog1/Y_2003_poly_181.tif...
Removing gs://tmpmarcog1/Y_2003_poly_181_8obs.tif...                            
Removing gs://tmpmarcog1/Y_2003_poly_182.tif...                                 
Removing gs://tmpmarcog1/Y_2003_poly_183_8obs.tif...                            
Removing gs://tmpmarcog1/Y_2003_poly_182_8obs.tif...                            
Removing gs://tmpmarcog1/Y_2003_poly_183.tif...                                 
Removing gs://tmpmarcog1/Y_2003_poly_2.tif...                                   
Removing gs://tmpmarcog1/Y_2003_poly_1_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_20.tif...
Removing gs://tmpmarcog1/Y_2003_poly_23.tif...
Removing gs://tmpmarcog1/Y_2003_poly_26_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_24_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_29.tif...
Removing gs://tmpmarcog1/Y_2003_poly_25.tif...
Removing gs://tmpmarcog1/Y_2003_poly_20_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_21.tif...
Removing gs://tmpmarcog1/Y_2003_poly_21_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_22.tif...
Removing gs://tmpmarcog1/Y_2003_poly_23_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_26.tif...
Removing gs://tmpmarcog1/Y_2003_poly_22_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_31_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_24.tif...
Removing gs://tmpmarcog1/Y_2003_poly_25_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_32.tif...
Removing gs://tmpmarcog1/Y_2003_poly_2_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_31.tif...
Removing gs://tmpmarcog1/Y_2003_poly_29_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_32_8obs.tif...                             
Removing gs://tmpmarcog1/Y_2003_poly_33.tif...                                  
Removing gs://tmpmarcog1/Y_2003_poly_33_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_34.tif...                                  
Removing gs://tmpmarcog1/Y_2003_poly_34_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_36.tif...
Removing gs://tmpmarcog1/Y_2003_poly_36_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_35.tif...
Removing gs://tmpmarcog1/Y_2003_poly_35_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_39.tif...                                  
Removing gs://tmpmarcog1/Y_2003_poly_37.tif...
Removing gs://tmpmarcog1/Y_2003_poly_37_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_39_8obs.tif...                             
Removing gs://tmpmarcog1/Y_2003_poly_40.tif...
Removing gs://tmpmarcog1/Y_2003_poly_40_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_41.tif...                                  
Removing gs://tmpmarcog1/Y_2003_poly_41_8obs.tif...                             
Removing gs://tmpmarcog1/Y_2003_poly_42.tif...                                  
Removing gs://tmpmarcog1/Y_2003_poly_43.tif...
Removing gs://tmpmarcog1/Y_2003_poly_43_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_42_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_44.tif...                                  
Removing gs://tmpmarcog1/Y_2003_poly_44_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_46_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_48.tif...                                  
Removing gs://tmpmarcog1/Y_2003_poly_48_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_46.tif...                                  
Removing gs://tmpmarcog1/Y_2003_poly_49.tif...
Removing gs://tmpmarcog1/Y_2003_poly_50.tif...
Removing gs://tmpmarcog1/Y_2003_poly_49_8obs.tif...                             
Removing gs://tmpmarcog1/Y_2003_poly_51.tif...                                  
Removing gs://tmpmarcog1/Y_2003_poly_50_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_51_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_52.tif...
Removing gs://tmpmarcog1/Y_2003_poly_52_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_54.tif...
Removing gs://tmpmarcog1/Y_2003_poly_54_8obs.tif...                             
Removing gs://tmpmarcog1/Y_2003_poly_55.tif...
Removing gs://tmpmarcog1/Y_2003_poly_55_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_56.tif...                                  
Removing gs://tmpmarcog1/Y_2003_poly_59.tif...                                  
Removing gs://tmpmarcog1/Y_2003_poly_57.tif...
Removing gs://tmpmarcog1/Y_2003_poly_59_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_56_8obs.tif...                             
Removing gs://tmpmarcog1/Y_2003_poly_57_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_60.tif...                                  
Removing gs://tmpmarcog1/Y_2003_poly_60_8obs.tif...                             
Removing gs://tmpmarcog1/Y_2003_poly_61_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_61.tif...
Removing gs://tmpmarcog1/Y_2003_poly_62.tif...
Removing gs://tmpmarcog1/Y_2003_poly_62_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_64.tif...
Removing gs://tmpmarcog1/Y_2003_poly_63_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_64_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_63.tif...
Removing gs://tmpmarcog1/Y_2003_poly_68.tif...
Removing gs://tmpmarcog1/Y_2003_poly_68_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_69_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_70.tif...
Removing gs://tmpmarcog1/Y_2003_poly_69.tif...
Removing gs://tmpmarcog1/Y_2003_poly_70_8obs.tif...                             
Removing gs://tmpmarcog1/Y_2003_poly_73.tif...                                  
Removing gs://tmpmarcog1/Y_2003_poly_71.tif...                                  
Removing gs://tmpmarcog1/Y_2003_poly_71_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_73_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_74.tif...
Removing gs://tmpmarcog1/Y_2003_poly_74_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_75.tif...
Removing gs://tmpmarcog1/Y_2003_poly_75_8obs.tif...                             
Removing gs://tmpmarcog1/Y_2003_poly_76.tif...                                  
Removing gs://tmpmarcog1/Y_2003_poly_76_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_77.tif...
Removing gs://tmpmarcog1/Y_2003_poly_77_8obs.tif...                             
Removing gs://tmpmarcog1/Y_2003_poly_78.tif...                                  
Removing gs://tmpmarcog1/Y_2003_poly_78_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_79.tif...                                  
Removing gs://tmpmarcog1/Y_2003_poly_80.tif...
Removing gs://tmpmarcog1/Y_2003_poly_80_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_79_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_81.tif...                                  
Removing gs://tmpmarcog1/Y_2003_poly_81_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_82.tif...
Removing gs://tmpmarcog1/Y_2003_poly_82_8obs.tif...                             
Removing gs://tmpmarcog1/Y_2003_poly_83_8obs.tif...                             
Removing gs://tmpmarcog1/Y_2003_poly_84.tif...
Removing gs://tmpmarcog1/Y_2003_poly_83.tif...
Removing gs://tmpmarcog1/Y_2003_poly_85.tif...                                  
Removing gs://tmpmarcog1/Y_2003_poly_85_8obs.tif...                             
Removing gs://tmpmarcog1/Y_2003_poly_88.tif...                                  
Removing gs://tmpmarcog1/Y_2003_poly_88_8obs.tif...                             
Removing gs://tmpmarcog1/Y_2003_poly_89.tif...
Removing gs://tmpmarcog1/Y_2003_poly_89_8obs.tif...                             
Removing gs://tmpmarcog1/Y_2003_poly_90.tif...
Removing gs://tmpmarcog1/Y_2003_poly_90_8obs.tif...                             
Removing gs://tmpmarcog1/Y_2003_poly_91.tif...
Removing gs://tmpmarcog1/Y_2003_poly_92.tif...
Removing gs://tmpmarcog1/Y_2003_poly_92_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_91_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_93.tif...
Removing gs://tmpmarcog1/Y_2003_poly_93_8obs.tif...                             
Removing gs://tmpmarcog1/Y_2003_poly_94.tif...
Removing gs://tmpmarcog1/Y_2003_poly_94_8obs.tif...                             
Removing gs://tmpmarcog1/Y_2003_poly_95.tif...
Removing gs://tmpmarcog1/Y_2003_poly_95_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_96.tif...                                  
Removing gs://tmpmarcog1/Y_2003_poly_96_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_97.tif...
Removing gs://tmpmarcog1/Y_2004_poly_100.tif...                                 
Removing gs://tmpmarcog1/Y_2004_poly_100_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_99.tif...
Removing gs://tmpmarcog1/Y_2003_poly_98_8obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_97_8obs.tif...                             
Removing gs://tmpmarcog1/Y_2004_poly_100_10obs.tif...
Removing gs://tmpmarcog1/Y_2003_poly_99_8obs.tif...
Removing gs://tmpmarcog1/Y_2004_poly_101.tif...
Removing gs://tmpmarcog1/Y_2004_poly_101_8obs.tif...
Removing gs://tmpmarcog1/Y_2004_poly_10.tif...
Removing gs://tmpmarcog1/Y_2003_poly_98.tif...
Removing gs://tmpmarcog1/Y_2004_poly_1.tif...
Removing gs://tmpmarcog1/Y_2004_poly_101_10obs.tif...
Removing gs://tmpmarcog1/Y_2004_poly_102.tif...
Removing gs://tmpmarcog1/Y_2004_poly_102_10obs.tif...                           
Removing gs://tmpmarcog1/Y_2004_poly_102_8obs.tif...
Removing gs://tmpmarcog1/Y_2004_poly_103.tif...
Removing gs://tmpmarcog1/Y_2004_poly_103_10obs.tif...
Removing gs://tmpmarcog1/Y_2004_poly_103_8obs.tif...                            
Removing gs://tmpmarcog1/Y_2004_poly_104_10obs.tif...                           
Removing gs://tmpmarcog1/Y_2004_poly_104_8obs.tif...                            
Removing gs://tmpmarcog1/Y_2004_poly_104.tif...
Removing gs://tmpmarcog1/Y_2004_poly_108.tif...
Removing gs://tmpmarcog1/Y_2004_poly_107.tif...
Removing gs://tmpmarcog1/Y_2004_poly_107_10obs.tif...
Removing gs://tmpmarcog1/Y_2004_poly_108_10obs.tif...
Removing gs://tmpmarcog1/Y_2004_poly_107_8obs.tif...
Removing gs://tmpmarcog1/Y_2004_poly_108_8obs.tif...                            
Removing gs://tmpmarcog1/Y_2004_poly_109_10obs.tif...                           
Removing gs://tmpmarcog1/Y_2004_poly_109_8obs.tif...
Removing gs://tmpmarcog1/Y_2004_poly_10_10obs.tif...
Removing gs://tmpmarcog1/Y_2004_poly_109.tif...
Removing gs://tmpmarcog1/Y_2004_poly_10_8obs.tif...
Removing gs://tmpmarcog1/Y_2004_poly_11.tif...                                  
Removing gs://tmpmarcog1/Y_2004_poly_110.tif...                                 
Removing gs://tmpmarcog1/Y_2004_poly_110_8obs.tif...                            
Removing gs://tmpmarcog1/Y_2004_poly_110_10obs.tif...
Removing gs://tmpmarcog1/Y_2004_poly_111.tif...                                 
Removing gs://tmpmarcog1/Y_2004_poly_111_10obs.tif...                           
Removing gs://tmpmarcog1/Y_2004_poly_112.tif...                                 
Removing gs://tmpmarcog1/Y_2004_poly_111_8obs.tif...
Removing gs://tmpmarcog1/Y_2004_poly_112_8obs.tif...                            
Removing gs://tmpmarcog1/Y_2004_poly_112_10obs.tif...
Removing gs://tmpmarcog1/Y_2004_poly_113.tif...
Removing gs://tmpmarcog1/Y_2004_poly_113_10obs.tif...                           
Removing gs://tmpmarcog1/Y_2004_poly_114.tif...
Removing gs://tmpmarcog1/Y_2004_poly_114_10obs.tif...
Removing gs://tmpmarcog1/Y_2004_poly_113_8obs.tif...                            
Removing gs://tmpmarcog1/Y_2004_poly_114_8obs.tif...
Removing gs://tmpmarcog1/Y_2004_poly_115.tif...
Removing gs://tmpmarcog1/Y_2004_poly_115_10obs.tif...
Removing gs://tmpmarcog1/Y_2004_poly_115_8obs.tif...                            
Removing gs://tmpmarcog1/Y_2004_poly_116.tif...
Removing gs://tmpmarcog1/Y_2004_poly_116_10obs.tif...
Removing gs://tmpmarcog1/Y_2004_poly_116_8obs.tif...
Removing gs://tmpmarcog1/Y_2004_poly_117_10obs.tif...                           
Removing gs://tmpmarcog1/Y_2004_poly_117.tif...
Removing gs://tmpmarcog1/Y_2004_poly_117_8obs.tif...                            
Removing gs://tmpmarcog1/Y_2004_poly_118.tif...
Removing gs://tmpmarcog1/Y_2004_poly_118_10obs.tif...